In [19]:
import pandas as pd
import numpy_financial as npf
import numpy as np
import datetime
from re import sub
from decimal import Decimal

from config import getConfig
from calcIRR import calcIRR
from calcAnnualReturn import calcAnnualReturn

In [20]:
def formatValue(x):
    return Decimal(sub(r'[^\d.]', '', x))

In [21]:
def getDays(df):
    startDate = acctValues.iloc[-1]['date']
    startDate = pd.to_datetime(startDate).date()
    endDate = acctValues.iloc[0]['date']
    endDate = pd.to_datetime(endDate).date()

    diff = datetime.date.today()-startDate
    print("{:,} days between {} and {}".format(diff.days, startDate, endDate))
    return diff.days

In [22]:
def filterDates(df):
    df = df[df['date'] >= startDt]
    df = df[df['date']  <= endDt]
    return df

In [23]:
def getDeposits(account):
    loc = cfg[account]['dataLoc']
    return pd.read_csv(loc+'deposits.csv')

In [24]:
def getValues(account):
    loc = cfg[account]['dataLoc']
    fileName = cfg[account]['fileName']
    return pd.read_csv(loc+fileName,
                       usecols=['date', 'value'])

In [25]:
def calcAnnualizedGrowth(irr):
    # Growth of 10k over a year at the computed rate
    PV = 1e4
    FV = npf.fv(irr, nper=365, pmt=0, pv=-PV)
    print("Annualized rate: {:.2%}".format(FV/PV -1))

In [26]:
cfg = getConfig()

In [27]:
etValues = getValues('ETrade')
etDeposits = getDeposits('ETrade')
spx = getValues('SPX')
msci = getValues('MSCI')

# Remove $ sign from account values
etValues['value'] = etValues['value'].apply(lambda x: formatValue(x))

In [28]:
# Make sure the starting dates are matched between accounts, deposits, benchmarks
startDt = etValues['date'].min()
endDt = etValues['date'].max()

etDeposits = filterDates(etDeposits)
spx = filterDates(spx)
msci = filterDates(msci)
msci.reset_index(drop=True, inplace=True)

In [30]:
# The 'initial account value' is basically the first deposit so add a row for it
d = {}
etValues.sort_values('date', ignore_index=True, inplace=True)
d['value'] = etValues.iloc[0]['value']
d['date'] = etValues.iloc[0]['date']

etDeposits.loc[len(etDeposits)] = d
etDeposits.sort_values('date', ignore_index=True, inplace=True)

In [31]:
# Ending value and date
endValue = float(etValues.iloc[-1]['value'])
endDate = etValues.iloc[-1]['date']
endDate = pd.to_datetime(endDate).date()

In [32]:
irr = calcIRR(etDeposits, endDate, endValue, 100)
calcAnnualizedGrowth(irr)

Total: 242,797

Annualized rate: 9.76%


##### Now the Benchmarks

In [33]:
# SPX
endValue = spx.iloc[-1]['value']
# 'calcIRR' requires a DF as input
initVal = pd.DataFrame([spx.iloc[0]])

irr = calcIRR(initVal, endDate, endValue, 5)
calcAnnualizedGrowth(irr)

Total: 4,765

Annualized rate: 8.26%


In [35]:
# MSCI
endValue = msci.iloc[-1]['value']
# 'calcIRR' requires a DF as input
initVal = pd.DataFrame([msci.iloc[0]])

irr = calcIRR(initVal, endDate, endValue, 2)
calcAnnualizedGrowth(irr)

Total: 133

Annualized rate: 5.65%


In [40]:
calcAnnualReturn(spx)

Year     Return
2021     26.9%
2022     -19.4%
2023     24.2%
